In [1]:
import pandas as pd
import json
import numpy as np
import scipy
import scipy.optimize

In [2]:
def eng_by_coeff(df, coeff):
    res = None
    for entry, val in coeff.items():
        if res is None: res = val * df.loc[entry]
        else: res += val * df.loc[entry]
    return res

## Read data

In [3]:
with open("../MN15_REF.json") as f:
    MN15_REF = json.load(f)
with open("../GMTKN55_REF.json") as f:
    GMTKN55_REF = json.load(f)

In [4]:
df_GMTKN55_b3lyp = pd.read_csv("../GMTKN55_b3lyp.csv", header=[0, 1], index_col=[0])
df_MN15_b3lyp = pd.read_csv("../MN15_b3lyp.csv", header=[0, 1], index_col=[0])

In [5]:
df_react_GMTKN55 = pd.DataFrame(
    index=list(df_GMTKN55_b3lyp.index) + ["ref"],
    columns=pd.MultiIndex.from_tuples([v.split("/") for v in GMTKN55_REF.keys()]),
    data=0.0)
for entry, info in GMTKN55_REF.items():
    dataset, idx = entry.split("/")
    for syst, stoi in zip(info["systems"], info["stoichiometry"]):
        dat, syst = syst.split("/")
        df_react_GMTKN55[(dataset, idx)] += stoi * df_GMTKN55_b3lyp[dat][syst]
        df_react_GMTKN55.loc["ref", (dataset, idx)] = info["ref"]

In [6]:
df_react_MN15 = pd.DataFrame(
    index=list(df_MN15_b3lyp.index) + ["ref", "bond"],
    columns=pd.MultiIndex.from_tuples([v.split("/") for v in MN15_REF.keys()]),
    data=0.0)
for entry, info in MN15_REF.items():
    dataset, idx = entry.split("/")
    for syst, stoi in zip(info["systems"], info["stoichiometry"]):
        df_react_MN15[(dataset, idx)] += stoi * df_MN15_b3lyp[dataset][syst]
        df_react_MN15.loc["ref", (dataset, idx)] = info["ref"]
        df_react_MN15.loc["bond", (dataset, idx)] = info["bond"]

## Existing Data

In [7]:
para_exist = {
    "B3LYP": {
        "eng_noxc": 1,
        "eng_exx_HF": 0.2,
        "eng_purexc_LDA_X": 0.08,
        "eng_purexc_GGA_X_B88": 0.72,
        "eng_purexc_LDA_C_VWN_RPA": 0.19,
        "eng_purexc_GGA_C_LYP": 0.81
    },
    "B3LYP5": {
        "eng_noxc": 1,
        "eng_exx_HF": 0.2,
        "eng_purexc_LDA_X": 0.08,
        "eng_purexc_GGA_X_B88": 0.72,
        "eng_purexc_LDA_C_VWN": 0.19,
        "eng_purexc_GGA_C_LYP": 0.81
    },
    "XYG3": {
        "eng_noxc": 1,
        "eng_exx_HF": 0.8033,
        "eng_purexc_LDA_X": -0.0140,
        "eng_purexc_GGA_X_B88": 0.2107,
        "eng_purexc_GGA_C_LYP": 0.6789,
        "eng_corr_MP2": 0.3211
    },
    "revXYG3": {
        "eng_noxc": 1,
        "eng_exx_HF": 0.9196,
        "eng_purexc_LDA_X": -0.0222,
        "eng_purexc_GGA_X_B88": 0.1026,
        "eng_purexc_GGA_C_LYP": 0.6059,
        "eng_corr_MP2": 0.3941
    },
    "revXYGJ-OS": {
        "eng_noxc": 1,
        "eng_exx_HF": 0.8877,
        "eng_purexc_LDA_X": 0.1123,
        "eng_purexc_LDA_C_VWN_RPA": -0.0697,
        "eng_purexc_GGA_C_LYP": 0.6167,
        "eng_corr_MP2_OS": 0.5485,
    },
    "XYG5": {
        "eng_noxc": 1,
        "eng_exx_HF": 0.9150,
        "eng_purexc_LDA_X": 0.0612,
        "eng_purexc_GGA_X_B88": 0.0238,
        "eng_purexc_LDA_C_VWN_RPA": 0,
        "eng_purexc_GGA_C_LYP": 0.4957,
        "eng_corr_MP2_OS": 0.4548,
        "eng_corr_MP2_SS": 0.2764,
    },
    "XYG6": {
        "eng_noxc": 1,
        "eng_exx_HF": 0.9105,
        "eng_purexc_LDA_X": 0.1576,
        "eng_purexc_GGA_X_B88": -0.0681,
        "eng_purexc_LDA_C_VWN_RPA": 0.1800,
        "eng_purexc_GGA_C_LYP": 0.2244,
        "eng_corr_MP2_OS": 0.4695,
        "eng_corr_MP2_SS": 0.2426,
    },
    "XYG7": {
        "eng_noxc": 1,
        "eng_exx_HF": 0.8971,
        "eng_purexc_LDA_X": 0.2055,
        "eng_purexc_GGA_X_B88": -0.1408,
        "eng_purexc_LDA_C_VWN_RPA": 0.4056,
        "eng_purexc_GGA_C_LYP": 0.1159,
        "eng_corr_MP2_OS": 0.4052,
        "eng_corr_MP2_SS": 0.2589,
    },
}

## Useful functions

In [8]:
def err_wtmad2(para):
    r = eng_by_coeff(df_react_GMTKN55, para) * 627.51
    d = r - df_react_GMTKN55.loc["ref"]

    err = 0
    for dataset in d.index.levels[0]:
        err += 56.84 * d[dataset].abs().sum() / df_react_GMTKN55.loc["ref"][dataset].abs().mean()
    err /= df_react_GMTKN55.columns.size
    return err

In [9]:
def err_our(para, to_list=False):
    err_wt = err_wtmad2(para)
    ret = {"GMTKN55": err_wt}
    
    r = eng_by_coeff(df_react_MN15, para) * 627.51
    d = (r - df_react_MN15.loc["ref"]) / df_react_MN15.loc["bond"]

    for dataset in d.index.levels[0]:
        ret[dataset] = d[dataset].abs().mean()
    ret = pd.Series(ret)
    
    if to_list:
        return ret
    else:
        ret /= 25
        ret["GMTKN55"]     *= 12
        ret["MR-MGM-BE4"]  *= 3
        ret["MR-MGN-BE17"] *= 2
        ret["MR-TM-BE13"]  *= 3
        ret["SR-MGM-BE9"]  *= 2
        ret["SR-TM-BE17"]  *= 3
        return ret.sum()

In [10]:
err_wtmad2(para_exist["B3LYP"])

15.901263668183562

In [11]:
err_wtmad2(para_exist["XYG7"])

2.008436604756705

In [12]:
def get_para_xyg7(arr):
    return {
        "eng_noxc": 1,
        "eng_exx_HF": arr[0],
        "eng_purexc_LDA_X": arr[1],
        "eng_purexc_GGA_X_B88": arr[2],
        "eng_purexc_LDA_C_VWN_RPA": arr[3],
        "eng_purexc_GGA_C_LYP": arr[4],
        "eng_corr_MP2_OS": arr[5],
        "eng_corr_MP2_SS": arr[6],
    }
    

In [13]:
def str_arr(arr):
    return "[" + ", ".join([f"{i:.6f}" for i in arr]) + "]"

## Optimize: without cr

### XYG7 reoptimize

In [14]:
def get_para_xyg7(arr):
    c_x, c_s, c_b88, c_vwn, c_lyp, c_os, c_ss = arr
    return {
        "eng_noxc": 1,
        "eng_exx_HF": c_x,
        "eng_purexc_LDA_X": c_s,
        "eng_purexc_GGA_X_B88": c_b88,
        "eng_purexc_LDA_C_VWN": c_vwn,
        "eng_purexc_GGA_C_LYP": c_lyp,
        "eng_corr_MP2_OS": c_os,
        "eng_corr_MP2_SS": c_ss,
    }

In [15]:
obj = lambda arr: err_wtmad2(get_para_xyg7(arr))
res = scipy.optimize.minimize(
    obj,
    [0.8865, 0.2162, -0.1432, 0.4658, 0.0739, 0.3870, 0.2595],
    method="L-BFGS-B",
    options={"ftol": 1e-5},
    callback=lambda arr: print(str_arr(arr), f"{obj(arr):.6f}"),
)

[0.889795, 0.221816, -0.140156, 0.466689, 0.075388, 0.388785, 0.260960] 2.072687
[0.888990, 0.221251, -0.140540, 0.466808, 0.075414, 0.389434, 0.261179] 2.064998
[0.887542, 0.220760, -0.140900, 0.467391, 0.075791, 0.391640, 0.262012] 2.055536
[0.886519, 0.220656, -0.140680, 0.468264, 0.076303, 0.394651, 0.263103] 2.050437
[0.887225, 0.220071, -0.139725, 0.468815, 0.076146, 0.395598, 0.262886] 2.046310
[0.889030, 0.217743, -0.137206, 0.470352, 0.075515, 0.398234, 0.262124] 2.038565
[0.891671, 0.213355, -0.133393, 0.472911, 0.074409, 0.402853, 0.260896] 2.030757
[0.892924, 0.209352, -0.130128, 0.475631, 0.073576, 0.408202, 0.260663] 2.026455
[0.893474, 0.208059, -0.129366, 0.476397, 0.073233, 0.409662, 0.260778] 2.026260
[0.893660, 0.208666, -0.130200, 0.475862, 0.073222, 0.408563, 0.261164] 2.026030
[0.894273, 0.208046, -0.130034, 0.476207, 0.072902, 0.409251, 0.261703] 2.025716
[0.897473, 0.205567, -0.130546, 0.477247, 0.071179, 0.411294, 0.265255] 2.024338
[0.898723, 0.203801, -0.1299

In [16]:
err_our(get_para_xyg7(res.x), True)

GMTKN55         2.021747
MR-MGM-BE4     19.112914
MR-MGN-BE17     2.595663
MR-TM-BE13     12.384993
SR-MGM-BE9      2.081580
SR-TM-BE17      6.909309
dtype: float64

In [17]:
err_our(para_exist["XYG7"], True)

GMTKN55         2.008437
MR-MGM-BE4     17.806289
MR-MGN-BE17     2.632873
MR-TM-BE13     12.119385
SR-MGM-BE9      2.548175
SR-TM-BE17      6.754149
dtype: float64

### XYG6 reoptimize

In [18]:
def get_para_xyg6(arr):
    c_x, c_b88, c_vwn, c_lyp, c_os, c_ss = arr
    return {
        "eng_noxc": 1,
        "eng_exx_HF": c_x,
        "eng_purexc_LDA_X": (1 - c_x - c_b88),
        "eng_purexc_GGA_X_B88": c_b88,
        "eng_purexc_LDA_C_VWN_RPA": c_vwn,
        "eng_purexc_GGA_C_LYP": c_lyp,
        "eng_corr_MP2_OS": c_os,
        "eng_corr_MP2_SS": c_ss,
    }

In [19]:
obj = lambda arr: err_wtmad2(get_para_xyg6(arr))
res = scipy.optimize.minimize(
    obj,
    [0.9105, -0.0681, 0.1800, 0.2244, 0.4695, 0.2426],
    method="L-BFGS-B",
    options={"ftol": 1e-5},
    callback=lambda arr: print(str_arr(arr), f"{obj(arr):.6f}"),
)

[0.912243, -0.065554, 0.180076, 0.223600, 0.468093, 0.241146] 2.163202
[0.910768, -0.061893, 0.182865, 0.224147, 0.468243, 0.239887] 2.154637
[0.903982, -0.050728, 0.193089, 0.226870, 0.469433, 0.236199] 2.142930
[0.900273, -0.049780, 0.199606, 0.230323, 0.464362, 0.233406] 2.132375
[0.897008, -0.050461, 0.204466, 0.233077, 0.457790, 0.231313] 2.128167
[0.895134, -0.049287, 0.205958, 0.233302, 0.455913, 0.230714] 2.127222
[0.890498, -0.047518, 0.208032, 0.232851, 0.450445, 0.230240] 2.126931
[0.892464, -0.048153, 0.206174, 0.232145, 0.453345, 0.231271] 2.125959
[0.893037, -0.048028, 0.205098, 0.231280, 0.454678, 0.232142] 2.125679
[0.893928, -0.047317, 0.205737, 0.229316, 0.455560, 0.235233] 2.125481
[0.893211, -0.045936, 0.209551, 0.227026, 0.453203, 0.239566] 2.124513
[0.893434, -0.045315, 0.212394, 0.224838, 0.451223, 0.243410] 2.124061
[0.893583, -0.044739, 0.215219, 0.222981, 0.449414, 0.246860] 2.123861
[0.893528, -0.044586, 0.216119, 0.222481, 0.448815, 0.247827] 2.123815
[0.893

In [20]:
err_our(get_para_xyg6(res.x), to_list=True)

GMTKN55         2.123444
MR-MGM-BE4     21.905722
MR-MGN-BE17     3.638219
MR-TM-BE13     12.695500
SR-MGM-BE9      1.733340
SR-TM-BE17      6.960193
dtype: float64

In [21]:
err_our(para_exist["XYG6"], True)

GMTKN55         2.181838
MR-MGM-BE4     22.850564
MR-MGN-BE17     3.972678
MR-TM-BE13     13.198827
SR-MGM-BE9      1.778138
SR-TM-BE17      7.225776
dtype: float64

### XYG7 (with M15)

In [22]:
obj = lambda arr: err_our(get_para_xyg7(arr))
res = scipy.optimize.minimize(
    obj,
    [0.8865, 0.2162, -0.1432, 0.4658, 0.0739, 0.3870, 0.2595],
    method="L-BFGS-B",
    options={"ftol": 1e-5},
    callback=lambda arr: print(str_arr(arr), f"{obj(arr):.6f}"),
)

[0.893550, 0.224605, -0.136477, 0.466756, 0.075120, 0.385231, 0.259364] 5.471884
[0.893077, 0.222517, -0.137541, 0.466323, 0.074446, 0.382378, 0.257921] 5.378454
[0.893744, 0.220375, -0.138420, 0.466042, 0.073857, 0.374125, 0.254016] 5.308049
[0.896452, 0.222098, -0.136369, 0.466193, 0.073852, 0.368439, 0.251621] 5.238056
[0.897145, 0.225606, -0.133136, 0.466989, 0.074523, 0.363001, 0.249445] 5.186455
[0.896063, 0.229309, -0.129545, 0.468187, 0.075452, 0.357076, 0.247125] 5.183938
[0.895399, 0.229543, -0.129129, 0.468467, 0.075615, 0.353656, 0.245638] 5.166270
[0.887270, 0.234429, -0.125011, 0.471107, 0.077879, 0.347550, 0.243420] 5.121307
[0.822245, 0.273262, -0.091622, 0.492503, 0.095993, 0.289627, 0.221643] 4.911240
[0.747819, 0.307581, -0.061094, 0.518604, 0.117388, 0.246834, 0.203650] 4.763234
[0.740630, 0.310590, -0.056968, 0.523087, 0.120440, 0.241014, 0.199203] 4.752571
[0.722947, 0.314162, -0.047927, 0.538245, 0.130221, 0.231731, 0.185772] 4.733939
[0.730602, 0.310684, -0.0512

In [23]:
err_our(get_para_xyg7(res.x), True)

GMTKN55        4.254253
MR-MGM-BE4     7.645300
MR-MGN-BE17    3.660645
MR-TM-BE13     5.762221
SR-MGM-BE9     2.161583
SR-TM-BE17     4.924480
dtype: float64

### XYG6 (with M15)

In [24]:
obj = lambda arr: err_our(get_para_xyg6(arr))
res = scipy.optimize.minimize(
    obj,
    [0.9105, -0.0681, 0.1800, 0.2244, 0.4695, 0.2426],
    method="L-BFGS-B",
    options={"ftol": 1e-5},
    callback=lambda arr: print(str_arr(arr), f"{obj(arr):.6f}"),
)

[0.926575, -0.060488, 0.175134, 0.218282, 0.444600, 0.230987] 6.065552
[0.914471, -0.088311, 0.181350, 0.230736, 0.410705, 0.224826] 5.528478
[0.879178, -0.099793, 0.200254, 0.245178, 0.369885, 0.215939] 5.268296
[0.852146, -0.090249, 0.219451, 0.254051, 0.349315, 0.203299] 5.113604
[0.746686, -0.031193, 0.333896, 0.308049, 0.263957, 0.140774] 4.741742
[0.739657, -0.026573, 0.342210, 0.311703, 0.253057, 0.133633] 4.705575
[0.745254, -0.023775, 0.338237, 0.308404, 0.260955, 0.135757] 4.685078
[0.746159, -0.023491, 0.337582, 0.307903, 0.262172, 0.136144] 4.683756
[0.747703, -0.017895, 0.338417, 0.306685, 0.265190, 0.134729] 4.675237
[0.745379, -0.014642, 0.341422, 0.307560, 0.263952, 0.132732] 4.675056
[0.745249, -0.014530, 0.339736, 0.306313, 0.264571, 0.133178] 4.673429
[0.747061, -0.014876, 0.334661, 0.302947, 0.267407, 0.134921] 4.672823
[0.741008, -0.015783, 0.327137, 0.297625, 0.265483, 0.135847] 4.667273
[0.740278, -0.015227, 0.327714, 0.297588, 0.265020, 0.135525] 4.667209
[0.740

In [25]:
err_our(get_para_xyg6(res.x), True)

GMTKN55        4.111681
MR-MGM-BE4     8.298238
MR-MGN-BE17    3.893899
MR-TM-BE13     5.381641
SR-MGM-BE9     2.188687
SR-TM-BE17     4.709890
dtype: float64

## Optimize: with cr

### Our Model: XYG6+1/cr

In [26]:
def get_para_cr(arr):
    c_x, c_b88, c_vwn, c_lyp, c_os, c_ss, c_cr = arr
    return {
        "eng_noxc": 1,
        "eng_exx_HF": c_x,
        "eng_purexc_LDA_X": (1 - c_x - c_b88),
        "eng_purexc_GGA_X_B88": c_b88,
        "eng_purexc_LDA_C_VWN_RPA": c_vwn,
        "eng_purexc_GGA_C_LYP": c_lyp,
        "eng_corr_MP2_OS": c_os * (1 - c_cr),
        "eng_corr_MP2_SS": c_ss * (1 - c_cr),
        "eng_corr_MP2CR_OS": c_os * c_cr,
        "eng_corr_MP2CR_SS": c_ss * c_cr,
    }

In [27]:
obj = lambda arr: err_our(get_para_cr(arr))
res = scipy.optimize.minimize(
    obj,
    [0.9105, -0.0681, 0.1800, 0.2244, 0.4695, 0.2426, 0.7],
    method="L-BFGS-B",
    options={"ftol": 1e-5},
    callback=lambda arr: print(str_arr(arr), f"{obj(arr):.6f}"),
)

[0.899658, -0.072373, 0.183045, 0.227762, 0.478606, 0.247490, 0.697940] 3.550672
[0.890048, -0.065840, 0.181199, 0.224042, 0.484784, 0.247645, 0.690711] 3.508918
[0.880375, -0.054375, 0.177077, 0.216910, 0.489671, 0.246053, 0.680689] 3.468873
[0.877868, -0.045584, 0.172851, 0.210671, 0.488050, 0.242787, 0.673985] 3.411851
[0.874487, -0.041648, 0.170841, 0.207671, 0.488661, 0.241867, 0.669616] 3.405798
[0.868980, -0.043300, 0.167357, 0.204430, 0.483210, 0.241870, 0.660313] 3.377336
[0.862963, -0.052050, 0.166830, 0.204897, 0.473784, 0.234658, 0.643912] 3.338836
[0.853711, -0.059303, 0.167508, 0.203877, 0.463117, 0.219856, 0.609198] 3.297333
[0.852470, -0.060124, 0.168007, 0.203863, 0.461975, 0.217235, 0.603191] 3.290269
[0.851581, -0.061029, 0.168683, 0.204002, 0.460751, 0.214444, 0.597198] 3.283117
[0.851546, -0.061062, 0.168713, 0.204008, 0.460703, 0.214325, 0.596939] 3.282793
[0.851546, -0.061062, 0.168713, 0.204008, 0.460703, 0.214325, 0.596938] 3.282791


In [28]:
err_our(get_para_cr(res.x), to_list=True)

GMTKN55        2.360751
MR-MGM-BE4     3.239840
MR-MGN-BE17    1.653429
MR-TM-BE13     7.395569
SR-MGM-BE9     1.974187
SR-TM-BE17     4.859773
dtype: float64

In [29]:
get_para_cr([0.840881, -0.032755, 0.170599, 0.196057, 0.457957, 0.199048, 0.590478])

{'eng_noxc': 1,
 'eng_exx_HF': 0.840881,
 'eng_purexc_LDA_X': 0.19187400000000002,
 'eng_purexc_GGA_X_B88': -0.032755,
 'eng_purexc_LDA_C_VWN_RPA': 0.170599,
 'eng_purexc_GGA_C_LYP': 0.196057,
 'eng_corr_MP2_OS': 0.18754346655400003,
 'eng_corr_MP2_SS': 0.08151453505600001,
 'eng_corr_MP2CR_OS': 0.270413533446,
 'eng_corr_MP2CR_SS': 0.117533464944}

### Our Model (GMTKN55 only)

In [30]:
obj = lambda arr: err_wtmad2(get_para_cr(arr))
res = scipy.optimize.minimize(
    obj,
    [0.9105, -0.0681, 0.1800, 0.2244, 0.4695, 0.2426, 0.7],
    method="L-BFGS-B",
    options={"ftol": 1e-5},
    callback=lambda arr: print(str_arr(arr), f"{obj(arr):.6f}"),
)

[0.898371, -0.074284, 0.182164, 0.227845, 0.478882, 0.249018, 0.698720] 2.425865
[0.893821, -0.070956, 0.182135, 0.226801, 0.484618, 0.251370, 0.697581] 2.352645
[0.881944, -0.053052, 0.179970, 0.219671, 0.502145, 0.257480, 0.693491] 2.183231
[0.879575, -0.038707, 0.177040, 0.213115, 0.512141, 0.261661, 0.690331] 2.086356
[0.878867, -0.026789, 0.173696, 0.206952, 0.520740, 0.266965, 0.686975] 2.068751
[0.880271, -0.030908, 0.174060, 0.208465, 0.517771, 0.266632, 0.687576] 2.061076
[0.883457, -0.035142, 0.172921, 0.208679, 0.515574, 0.270299, 0.686832] 2.050097
[0.889515, -0.038657, 0.169147, 0.206482, 0.514665, 0.279933, 0.683916] 2.033059
[0.898706, -0.037504, 0.161803, 0.199924, 0.518034, 0.303285, 0.675148] 2.008845
[0.898979, -0.033334, 0.160894, 0.197915, 0.518839, 0.313045, 0.670677] 2.002958
[0.897789, -0.029434, 0.162775, 0.198224, 0.517009, 0.317427, 0.667957] 1.998846
[0.895288, -0.023009, 0.167781, 0.200269, 0.511529, 0.324701, 0.663090] 1.992960
[0.894708, -0.018289, 0.1715

In [31]:
err_our(get_para_cr(res.x), to_list=True)

GMTKN55         1.976482
MR-MGM-BE4      7.133091
MR-MGN-BE17     2.190803
MR-TM-BE13     10.382881
SR-MGM-BE9      1.538939
SR-TM-BE17      5.783074
dtype: float64

## Optimize: with sIEPA

### Our Model: XYG6+1/sIEPA

In [32]:
def get_para_siepa(arr):
    c_x, c_b88, c_vwn, c_lyp, c_os, c_ss, c_siepa = arr
    return {
        "eng_noxc": 1,
        "eng_exx_HF": c_x,
        "eng_purexc_LDA_X": (1 - c_x - c_b88),
        "eng_purexc_GGA_X_B88": c_b88,
        "eng_purexc_LDA_C_VWN_RPA": c_vwn,
        "eng_purexc_GGA_C_LYP": c_lyp,
        "eng_corr_MP2_OS": c_os * (1 - c_siepa),
        "eng_corr_MP2_SS": c_ss * (1 - c_siepa),
        "eng_corr_SIEPA_OS": c_os * c_siepa,
        "eng_corr_SIEPA_SS": c_ss * c_siepa,
    }

In [33]:
obj = lambda arr: err_our(get_para_siepa(arr))
res = scipy.optimize.minimize(
    obj,
    [0.9105, -0.0681, 0.1800, 0.2244, 0.4695, 0.2426, 0.7],
    method="L-BFGS-B",
    options={"ftol": 1e-5},
    callback=lambda arr: print(str_arr(arr), f"{obj(arr):.6f}"),
)

[0.924312, -0.060613, 0.175777, 0.218850, 0.449568, 0.232392, 0.700795] 5.223028
[0.916559, -0.081291, 0.180610, 0.228265, 0.420194, 0.223922, 0.704791] 4.839432
[0.900308, -0.093174, 0.188557, 0.236615, 0.404785, 0.218726, 0.709016] 4.725992
[0.888372, -0.103718, 0.195292, 0.243791, 0.383430, 0.210217, 0.713460] 4.695793
[0.864662, -0.084573, 0.212481, 0.248386, 0.374145, 0.193285, 0.721553] 4.501217
[0.788330, -0.033356, 0.273918, 0.270049, 0.340424, 0.137501, 0.751118] 4.268063
[0.774452, -0.031578, 0.286595, 0.276795, 0.325137, 0.124921, 0.758703] 4.222986
[0.759819, -0.032258, 0.300354, 0.284002, 0.306730, 0.108117, 0.769638] 4.165738
[0.758826, -0.031459, 0.301238, 0.284272, 0.306331, 0.107286, 0.770118] 4.165635
[0.753449, -0.028156, 0.306104, 0.286009, 0.303078, 0.102344, 0.773063] 4.161597
[0.750531, -0.025824, 0.316760, 0.287633, 0.302965, 0.091697, 0.788387] 4.150832
[0.780794, -0.038866, 0.348533, 0.282641, 0.333982, 0.060521, 0.872773] 4.149852
[0.777438, -0.039758, 0.3309

In [34]:
err_our(get_para_siepa(res.x), to_list=True)

GMTKN55        2.943800
MR-MGM-BE4     3.624001
MR-MGN-BE17    2.417903
MR-TM-BE13     7.750066
SR-MGM-BE9     1.874516
SR-TM-BE17     5.608546
dtype: float64

### Our Model (GMTKN55 only)

In [35]:
obj = lambda arr: err_wtmad2(get_para_siepa(arr))
res = scipy.optimize.minimize(
    obj,
    [0.9105, -0.0681, 0.1800, 0.2244, 0.4695, 0.2426, 0.7],
    method="L-BFGS-B",
    options={"ftol": 1e-5},
    callback=lambda arr: print(str_arr(arr), f"{obj(arr):.6f}"),
)

[0.912926, -0.064220, 0.179776, 0.222990, 0.467568, 0.240472, 0.699955] 2.180265
[0.909127, -0.059877, 0.182261, 0.223205, 0.468828, 0.239117, 0.699811] 2.161923
[0.895187, -0.043019, 0.194360, 0.225387, 0.471040, 0.232468, 0.699122] 2.136638
[0.893037, -0.043696, 0.197287, 0.227285, 0.465393, 0.229914, 0.698980] 2.124439
[0.886132, -0.042915, 0.206052, 0.232157, 0.452076, 0.223950, 0.698508] 2.118717
[0.887765, -0.044126, 0.203971, 0.231227, 0.455219, 0.225849, 0.698614] 2.115861
[0.887576, -0.044201, 0.204210, 0.231378, 0.454860, 0.225759, 0.698599] 2.115822
[0.887268, -0.044321, 0.204549, 0.231562, 0.454611, 0.225828, 0.698570] 2.115740
[0.887177, -0.044166, 0.204629, 0.231562, 0.454632, 0.225779, 0.698565] 2.115724


In [36]:
err_our(get_para_siepa(res.x), to_list=True)

GMTKN55         2.115724
MR-MGM-BE4     15.616177
MR-MGN-BE17     3.058427
MR-TM-BE13     12.261220
SR-MGM-BE9      1.705065
SR-TM-BE17      6.772356
dtype: float64

## Optimize: with IEPA

### Our Model: XYG6+1/IEPA

In [37]:
def get_para_iepa(arr):
    c_x, c_b88, c_vwn, c_lyp, c_os, c_ss, c_iepa = arr
    return {
        "eng_noxc": 1,
        "eng_exx_HF": c_x,
        "eng_purexc_LDA_X": (1 - c_x - c_b88),
        "eng_purexc_GGA_X_B88": c_b88,
        "eng_purexc_LDA_C_VWN_RPA": c_vwn,
        "eng_purexc_GGA_C_LYP": c_lyp,
        "eng_corr_MP2_OS": c_os * (1 - c_iepa),
        "eng_corr_MP2_SS": c_ss * (1 - c_iepa),
        "eng_corr_IEPA_OS": c_os * c_iepa,
        "eng_corr_IEPA_SS": c_ss * c_iepa,
    }

In [38]:
obj = lambda arr: err_our(get_para_iepa(arr))
res = scipy.optimize.minimize(
    obj,
    [0.9105, -0.0681, 0.1800, 0.2244, 0.4695, 0.2426, 0.7],
    method="L-BFGS-B",
    options={"ftol": 1e-5},
    callback=lambda arr: print(str_arr(arr), f"{obj(arr):.6f}"),
)

[0.926941, -0.057065, 0.174871, 0.217162, 0.446896, 0.230468, 0.700478] 5.276477
[0.925248, -0.059253, 0.175721, 0.218389, 0.441074, 0.228158, 0.701064] 5.229288
[0.911543, -0.071859, 0.181481, 0.226146, 0.424887, 0.222827, 0.703367] 5.047758
[0.874379, -0.086916, 0.198243, 0.241933, 0.387637, 0.206128, 0.708771] 4.880349
[0.815773, -0.051674, 0.238590, 0.257497, 0.346874, 0.165203, 0.717521] 4.486291
[0.757430, -0.012816, 0.286938, 0.277038, 0.308466, 0.119689, 0.729137] 4.276690
[0.737102, 0.005080, 0.307795, 0.285307, 0.298796, 0.102510, 0.735336] 4.240435
[0.732995, 0.012288, 0.315889, 0.286450, 0.299194, 0.101431, 0.743486] 4.238081
[0.738504, 0.007791, 0.310431, 0.284500, 0.301891, 0.105313, 0.741212] 4.233081
[0.741960, 0.004386, 0.307454, 0.283743, 0.303163, 0.107184, 0.740722] 4.229327
[0.742877, 0.003130, 0.306248, 0.283276, 0.303387, 0.107644, 0.741370] 4.227521
[0.743794, 0.002615, 0.305892, 0.282280, 0.304231, 0.108760, 0.745871] 4.226111
[0.738250, 0.002667, 0.309494, 0.2

In [39]:
err_our(get_para_iepa(res.x), to_list=True)

GMTKN55        4.405915
MR-MGM-BE4     2.622221
MR-MGN-BE17    3.968204
MR-TM-BE13     5.304111
SR-MGM-BE9     2.026431
SR-TM-BE17     4.592854
dtype: float64

### Our Model (GMTKN55 only)

In [40]:
obj = lambda arr: err_wtmad2(get_para_iepa(arr))
res = scipy.optimize.minimize(
    obj,
    [0.9105, -0.0681, 0.1800, 0.2244, 0.4695, 0.2426, 0.7],
    method="L-BFGS-B",
    options={"ftol": 1e-5},
    callback=lambda arr: print(str_arr(arr), f"{obj(arr):.6f}"),
)

[0.916744, -0.057785, 0.178592, 0.220256, 0.464041, 0.237461, 0.699021] 2.905162
[0.906019, -0.045452, 0.181416, 0.218616, 0.464232, 0.234847, 0.694561] 2.821621
[0.872662, -0.016777, 0.191805, 0.217691, 0.464704, 0.229339, 0.680776] 2.724588
[0.873923, -0.019509, 0.195239, 0.221120, 0.457978, 0.227987, 0.677111] 2.692063
[0.868703, -0.022685, 0.214954, 0.236934, 0.431212, 0.221049, 0.655722] 2.635997
[0.860904, -0.014622, 0.222379, 0.240500, 0.427523, 0.220692, 0.643176] 2.610978
[0.856659, -0.005605, 0.229160, 0.244740, 0.427281, 0.228145, 0.614864] 2.580238
[0.857372, -0.000145, 0.235334, 0.249930, 0.423981, 0.236650, 0.582411] 2.551047
[0.859601, 0.040740, 0.257923, 0.278580, 0.394088, 0.323245, 0.282020] 2.421172
[0.893139, 0.029176, 0.251220, 0.288968, 0.418526, 0.328477, 0.196956] 2.318054
[0.905548, 0.005810, 0.233491, 0.289185, 0.428103, 0.308957, 0.179569] 2.242772
[0.906742, -0.014498, 0.194146, 0.282442, 0.431582, 0.289487, 0.086704] 2.154706
[0.896073, -0.020284, 0.186789,

In [41]:
err_our(get_para_iepa(res.x), to_list=True)

GMTKN55         2.114189
MR-MGM-BE4     20.924739
MR-MGN-BE17     3.434654
MR-TM-BE13     12.642269
SR-MGM-BE9      1.725337
SR-TM-BE17      6.924705
dtype: float64